In [164]:
import os
import cPickle as pickle
import configparser
import codecs
from distutils.util import strtobool
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

import sys
sys.path.append("../../MyPythonModule")
import mymodule
sys.path.append("../../Interactive_Graph_Visualizer/networkx-master")
import networkx as nx

In [170]:
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [172]:
"""指数で正規化したユークリッド距離"""
def compare4_2(p,q):
    weight = np.exp(-((p-q)**2).sum())
    return weight

In [173]:
def suffix_generator_root(search_word,max_page,add_childs,append):
    suffix = "_" + search_word
    suffix += "_" + unicode(max_page)
    if add_childs:
        suffix += "_add_childs"
    if append:
        suffix += "_append"
    return suffix

In [174]:
def suffix_generator(target=None,is_largest=False):
    suffix = ""
    if target != None:
        suffix += "_" + target
    if is_largest == True:
        suffix += "_largest"
    return suffix

In [175]:
search_word = "Test"
max_page = 10
add_childs = True
append = False
save_dir = "../Data/Search"
root_dir = save_dir + suffix_generator_root(search_word,max_page,add_childs,append)

is_largest = True
target = "myexttext"
K = 10
exp_name = "K" + unicode(K) + suffix_generator(target,is_largest)
exp_dir = os.path.join(root_dir,exp_name)
nx_dir = os.path.join(exp_dir,"nx_datas")

size = 100
exp_name_new = "D" + str(size) + suffix_generator(target,is_largest)
exp_dir_new = os.path.join(root_dir,exp_name_new)
nx_dir_new = os.path.join(exp_dir_new,"nx_datas")

In [176]:
if not os.path.exists(nx_dir_new):
    os.mkdir(nx_dir_new)

In [177]:
weights_list = []
weights_list_new = []
euclids = []
comp_func_name = "comp4_2"
G_path = "G_with_params_" + comp_func_name + ".gpkl"

In [178]:
with open(os.path.join(exp_dir,"instance.pkl"),'r') as fi:
    lda = pickle.load(fi)
with open(os.path.join(nx_dir,G_path),'r') as fi:
    G = pickle.load(fi)
with open(os.path.join(exp_dir_new,"doc2vec.pkl"),'rb') as fi:
    doc2vec_vectors = pickle.load(fi)

In [179]:
edges = G.edge
edges

{0: {11: {'weight': 0.35491573767099976},
  12: {'weight': 0.73905340895447713},
  13: {'weight': 0.86799707748927746},
  14: {'weight': 0.67233278811315134},
  16: {'weight': 0.47164812128681638},
  17: {'weight': 0.7810720628495238},
  18: {'weight': 0.78297359570848668},
  19: {'weight': 0.7727786796108117},
  20: {'weight': 0.37158674768559441},
  21: {'weight': 0.92355050127466465},
  22: {'weight': 0.75208237908875919},
  24: {'weight': 0.35513119284848693},
  25: {'weight': 0.65264089322309926},
  26: {'weight': 0.70191312769499892},
  27: {'weight': 0.91901443081352086}},
 2: {28: {'weight': 0.44736480413431307},
  29: {'weight': 0.444568041417777},
  31: {'weight': 0.59245830711492231}},
 3: {9: {'weight': 0.77711656171729204},
  32: {'weight': 0.53546233976420021},
  34: {'weight': 0.66010867321432587},
  35: {'weight': 0.5363941852381976},
  36: {'weight': 0.54802897651210725},
  38: {'weight': 0.67788170138369364},
  40: {'weight': 0.47039789450603375},
  41: {'weight': 0.7

In [180]:
nodes = G.node
nodes

{0: {'a_score': 8.561922690394718e-16,
  'color': '#00A0C1',
  'from_hosts': {u'time-space.kddi.com': 3, u'www.kddi.com': 1},
  'h_score': 4.999029713779984e-15,
  'to_hosts': {u'onlineshop.au.com': 1,
   u'star.auone.jp': 1,
   u'time-space.kddi.com': 5,
   u'wowma.jp': 1,
   u'www.kddi.com': 7},
  'topic': 5},
 2: {'a_score': 0.0,
  'color': '#0086D1',
  'from_hosts': {},
  'h_score': 0.014396614118132653,
  'to_hosts': {u'aiphone.co.jp': 1,
   u'itunes.apple.com': 1,
   u'support.apple.com': 1},
  'topic': 6},
 3: {'a_score': 0.0,
  'color': '#00479D',
  'from_hosts': {},
  'h_score': 0.1667736070325817,
  'to_hosts': {u'discussionsjapan.apple.com': 2, u'www.apple.com': 13},
  'topic': 7},
 4: {'a_score': 0.0,
  'color': '#FCC800',
  'from_hosts': {},
  'h_score': 0.0010197900420815058,
  'to_hosts': {u'aiphone.co.jp': 1, u'softbankmobile.co.jp': 1},
  'topic': 1},
 5: {'a_score': 2.4098746315893005e-16,
  'color': '#0086D1',
  'from_hosts': {u'time-space.kddi.com': 1},
  'h_score':

In [181]:
for i,i_node in enumerate(tqdm(nodes)):
    p_dst = doc2vec_vectors[i_node]
    for j,j_node in enumerate(nodes):
        if not j == i:
            q_dst = doc2vec_vectors[j_node]
            weight = cos_sim(p_dst,q_dst)
            weights_list.append(weight)#ヒストグラム作成用

100%|██████████| 43/43 [00:00<00:00, 685.48it/s]


In [182]:
len(weights_list)

1806

In [183]:
max = np.array(weights_list).max()
max

0.92199254

In [184]:
min = np.array(weights_list).min()
min

0.1341137

In [185]:
for node_no,link_node_nos in edges.items():
    p_dst = doc2vec_vectors[node_no]
    """類似度による重みの算出"""
    for link_node_no in link_node_nos.keys():
        q_dst = doc2vec_vectors[link_node_no]
        weight = cos_sim(p_dst,q_dst)
        weight = (weight - min)/(max - min)
        if weight == 0:
            weight = 0.001
        edges[node_no][link_node_no]["weight"] = weight

In [186]:
edges

{0: {11: {'weight': 0.49961248},
  12: {'weight': 0.47998479},
  13: {'weight': 0.32645592},
  14: {'weight': 0.45838967},
  16: {'weight': 0.41636637},
  17: {'weight': 0.54705793},
  18: {'weight': 0.46663508},
  19: {'weight': 0.45878243},
  20: {'weight': 0.4458937},
  21: {'weight': 0.1721084},
  22: {'weight': 0.34204951},
  24: {'weight': 0.50319284},
  25: {'weight': 0.43216383},
  26: {'weight': 0.44672605},
  27: {'weight': 0.44835702}},
 2: {28: {'weight': 0.58822834},
  29: {'weight': 0.67463875},
  31: {'weight': 0.39494073}},
 3: {9: {'weight': 0.64453572},
  32: {'weight': 0.38467708},
  34: {'weight': 0.64461696},
  35: {'weight': 0.39034313},
  36: {'weight': 0.48568183},
  38: {'weight': 0.50732535},
  40: {'weight': 0.6454007},
  41: {'weight': 0.66736752},
  42: {'weight': 0.69689602},
  43: {'weight': 0.53133094},
  44: {'weight': 0.63385361},
  47: {'weight': 0.45296311},
  48: {'weight': 0.55338126},
  49: {'weight': 0.52903706},
  50: {'weight': 0.45619696}},
 4

In [187]:
DEFAULT_WEIGHT = 0.5

In [189]:
nodes_lim = len(nodes)
nodes_lim

43

In [190]:
all_node_weights = np.ones((nodes_lim,nodes_lim))*DEFAULT_WEIGHT#除算の都合上，自分自身との類似度は1に
all_node_weights

array([[ 0.5,  0.5,  0.5, ...,  0.5,  0.5,  0.5],
       [ 0.5,  0.5,  0.5, ...,  0.5,  0.5,  0.5],
       [ 0.5,  0.5,  0.5, ...,  0.5,  0.5,  0.5],
       ..., 
       [ 0.5,  0.5,  0.5, ...,  0.5,  0.5,  0.5],
       [ 0.5,  0.5,  0.5, ...,  0.5,  0.5,  0.5],
       [ 0.5,  0.5,  0.5, ...,  0.5,  0.5,  0.5]])

In [191]:
doc2vec_vectors[0]

array([ 0.09728539,  0.11681181, -0.06479433,  0.09265144,  0.87168008,
        0.43172827,  0.28555998, -0.56806374,  0.62399924, -0.21992779,
        0.87188303, -0.30005711,  0.0418622 , -0.23005079, -0.56084543,
        0.57285142, -0.90125287,  0.21617565,  0.05607327, -0.81053352,
        0.20767464, -0.35054114, -0.21507274, -0.25569069,  0.47567216,
        0.00585597, -0.13244919,  0.65534228,  0.41328353,  0.55006796,
       -1.1059829 , -0.11195086, -0.87832701,  0.14115958,  0.46623084,
        1.10152054,  1.25085914, -0.20458646,  0.28048515, -0.11694862,
       -0.1672235 , -0.52451611,  1.39827645,  0.16238143, -0.1005708 ,
        0.37373319, -0.97177815,  0.4889718 ,  0.02024215, -0.56501257,
        0.32596564, -0.77922052,  0.44221625, -0.46073452,  0.63100719,
        0.49374112,  1.47673082, -0.96417469, -0.33211878,  0.3689599 ,
       -0.27646905,  0.5929358 , -0.05743116, -1.47552538, -0.46054065,
       -0.0986493 , -0.54994059, -0.16796967,  1.32938969, -0.32

In [192]:
for i,i_node in enumerate(tqdm(nodes)):
    p_dst = doc2vec_vectors[i_node]
    for j,j_node in enumerate(nodes):
        q_dst = doc2vec_vectors[j_node]
        weight = cos_sim(p_dst,q_dst)
        weight = (weight - min)/(max - min)
        weight_euclid = compare4_2(p_dst,q_dst)
        if weight == 0:
            weight = 0.001
        all_node_weights[i,j] = weight
        if not j == i:
            weights_list_new.append(weight)#ヒストグラム作成用
            euclids.append(weight_euclid)

100%|██████████| 43/43 [00:00<00:00, 387.52it/s]


In [193]:
all_node_weights

array([[ 1.09900963,  0.49354434,  0.55250967, ...,  0.29411235,
         0.4824844 ,  0.43197867],
       [ 0.49354434,  1.09900928,  0.71194136, ...,  0.31601304,
         0.5527482 ,  0.58892989],
       [ 0.55250967,  0.71194136,  1.09900951, ...,  0.45619696,
         0.57675642,  0.64878041],
       ..., 
       [ 0.29411235,  0.31601304,  0.45619696, ...,  1.09900951,
         0.37153214,  0.46203354],
       [ 0.4824844 ,  0.5527482 ,  0.57675642, ...,  0.37153214,
         1.09900928,  0.44868854],
       [ 0.43197867,  0.58892989,  0.64878041, ...,  0.46203354,
         0.44868854,  1.09900951]])

In [194]:
max_new = np.array(weights_list_new).max()
min_new = np.array(weights_list_new).min()

In [195]:
max_new

1.0

In [196]:
min_new

0.001

In [197]:
edges[2][28]

{'weight': 0.58822834}

In [198]:
cos_sim(doc2vec_vectors[2],doc2vec_vectors[28])

0.59756637

In [199]:
(cos_sim(doc2vec_vectors[2],doc2vec_vectors[28])-min)/(max-min)

0.58822834

In [200]:
all_node_weights[1,24]

0.58822834491729736

In [201]:
all_node_weights[24,1]

0.58822834491729736

In [202]:
compare4_2(doc2vec_vectors[2],doc2vec_vectors[28])

0.0

In [203]:
weights_list

[0.52296686,
 0.56942439,
 0.5307138,
 0.41537997,
 0.4353711,
 0.4325783,
 0.49407715,
 0.4566358,
 0.52774781,
 0.51228356,
 0.39132142,
 0.49526924,
 0.46215996,
 0.5651291,
 0.50176561,
 0.49557868,
 0.48542392,
 0.26971427,
 0.40360728,
 0.53056872,
 0.47460645,
 0.48607972,
 0.48736471,
 0.48582587,
 0.53004354,
 0.56015092,
 0.31620738,
 0.61379224,
 0.37173739,
 0.43379411,
 0.30718392,
 0.50740665,
 0.61415011,
 0.41189188,
 0.34468067,
 0.43852627,
 0.57430708,
 0.48230988,
 0.51400834,
 0.36583862,
 0.51425296,
 0.47446057,
 0.52296686,
 0.69503725,
 0.59725887,
 0.5918805,
 0.89470977,
 0.87948197,
 0.66692126,
 0.47169951,
 0.40424949,
 0.64156955,
 0.38915402,
 0.40376875,
 0.3231627,
 0.52221471,
 0.59712768,
 0.70461226,
 0.3602275,
 0.35914144,
 0.47158551,
 0.25505903,
 0.48901898,
 0.54233307,
 0.44670695,
 0.59756637,
 0.66564733,
 0.44527915,
 0.3027766,
 0.64646947,
 0.33678004,
 0.47097215,
 0.51240742,
 0.55851197,
 0.52568728,
 0.57960361,
 0.62755722,
 0.74228

In [204]:
weights_list_new

[0.49354434,
 0.55250967,
 0.50337702,
 0.35699177,
 0.38236514,
 0.37882042,
 0.45687664,
 0.40935493,
 0.49961248,
 0.47998479,
 0.32645592,
 0.45838967,
 0.41636637,
 0.54705793,
 0.46663508,
 0.45878243,
 0.4458937,
 0.1721084,
 0.34204951,
 0.50319284,
 0.43216383,
 0.44672605,
 0.44835702,
 0.44640386,
 0.50252628,
 0.54073948,
 0.23111887,
 0.6088227,
 0.30159926,
 0.38036355,
 0.21966603,
 0.47379485,
 0.60927689,
 0.35256457,
 0.26725805,
 0.38636976,
 0.55870694,
 0.44194126,
 0.48217392,
 0.29411235,
 0.4824844,
 0.43197867,
 0.49354434,
 0.71194136,
 0.58783805,
 0.58101165,
 0.96537179,
 0.94604421,
 0.6762557,
 0.42847425,
 0.34286463,
 0.64407849,
 0.32370499,
 0.34225446,
 0.23994678,
 0.49258968,
 0.58767152,
 0.72409427,
 0.28699055,
 0.28561211,
 0.42832956,
 0.15350753,
 0.45045665,
 0.51812452,
 0.39675292,
 0.58822834,
 0.67463875,
 0.39494073,
 0.21407212,
 0.6502977,
 0.25723034,
 0.42755106,
 0.48014197,
 0.53865927,
 0.49699718,
 0.56542945,
 0.62629366,
 0.77

In [205]:
len(weights_list)

1806

In [206]:
len(weights_list_new)

1806

In [207]:
with open(os.path.join(nx_dir_new,"G_with_params_cos_sim.gpkl"),'w') as fo:
    pickle.dump(G,fo)
with open(os.path.join(nx_dir_new,"all_node_weights_cos_sim.gpkl"),'w') as fo:
    pickle.dump(all_node_weights,fo)

In [209]:
"""weight（全ノード間の距離）のヒストグラム作成"""
fig_w = plt.figure()
ax = fig_w.add_subplot(1,1,1)
weights_array = np.array(weights_list_new,dtype=np.float)
ax.hist(weights_array,bins=100)
plt.text(0.5, 0.9, "max="+"{0:.3f}".format(weights_array.max()), transform=ax.transAxes)
plt.text(0.5, 0.85, "min="+"{0:.3g}".format(weights_array.min()), transform=ax.transAxes)
plt.text(0.5, 0.80, "num="+str(len(weights_list_new)), transform=ax.transAxes)
fig_w.show()
fig_w.savefig(os.path.join(nx_dir_new,"cos_sim_hist.png"))

In [216]:
fig_w = plt.figure()
ax = fig_w.add_subplot(1,1,1)
weights_array = np.array(euclids,dtype=np.float)
ax.hist(weights_array,bins=100)
# plt.text(0.5, 0.9, "max="+"{0:.3f}".format(weights_array.max()), transform=ax.transAxes)
plt.text(0.5, 0.9, "max="+str(weights_array.max()), transform=ax.transAxes)
# plt.text(0.5, 0.85, "min="+"{0:.3g}".format(weights_array.min()), transform=ax.transAxes)
plt.text(0.5, 0.85, "min="+str(weights_array.min()), transform=ax.transAxes)
plt.text(0.5, 0.80, "num="+str(len(euclids)), transform=ax.transAxes)
plt.xlim([weights_array.min(),weights_array.max()])
fig_w.show()
fig_w.savefig(os.path.join(nx_dir_new,"comp4_2_hist.png"))

In [211]:
weights_list.count(1)

0

In [212]:
weights_list_new.count(1)

2

In [213]:
euclids

[9.2714508e-38,
 4.4329557e-16,
 1.8866184e-20,
 0.0,
 0.0,
 0.0,
 3.371204e-17,
 8.4990241e-27,
 0.0,
 0.0,
 0.0,
 1.2649497e-20,
 4.6340797e-34,
 0.0,
 0.0,
 0.0,
 0.0,
 5.3907672e-40,
 0.0,
 6.8071078e-14,
 9.0964248e-31,
 1.3035936e-16,
 0.0,
 0.0,
 7.8316651e-28,
 2.5739317e-15,
 2.9177491e-22,
 2.3723289e-12,
 0.0,
 0.0,
 0.0,
 3.0815646e-25,
 4.4123629e-13,
 7.0770219e-20,
 1.8655441e-33,
 0.0,
 1.5121142e-16,
 4.0429573e-15,
 1.1984261e-19,
 0.0,
 2.0578408e-14,
 1.0748334e-24,
 9.2714508e-38,
 2.1160264e-27,
 9.8009202e-34,
 0.0,
 3.1694647e-26,
 2.3743657e-16,
 1.2008638e-29,
 2.6624671e-44,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.4012985e-45,
 1.4720705e-36,
 0.0,
 0.0,
 1.3118003e-30,
 5.9260912e-42,
 0.0,
 1.1611752e-31,
 0.0,
 0.0,
 0.0,
 5.8193481e-38,
 2.1255781e-37,
 1.9087417e-34,
 9.9927445e-33,
 7.1372674e-34,
 7.0064923e-45,
 1.8871286e-41,
 1.0593816e-42,
 0.0,
 5.4422246e-36,
 3.5443269e-34,
 4.4329557e-16,
 2.1160264e-27,
 1.35